## 일일 박스오피스 데이터 & 영화정보 merge

- 응답구조
    - boxofficeType :문자열	박스오피스 종류
    - showRange : 	문자열	박스오피스 조회 일자를 출력
    - rnum	문자열	순번을 출력
    - rank	문자열	해당일자의 박스오피스 순위를 출력
    - rankInten	문자열	전일대비 순위의 증감분을 출력
    - rankOldAndNew	문자열	랭킹에 신규진입여부를 출력
    - “OLD” : 기존 , “NEW” : 신규
    - movieCd: 문자열	영화의 대표코드를 출력
    - movieNm : 문자열	영화명(국문)을 출력
    - openDt : 문자열	영화의 개봉일을 출력
    - salesAmt : 문자열	해당일의 매출액을 출력
    - salesShare : 문자열	해당일자 상영작의 매출총액 대비 해당 영화의 매출비율을 출력
    - salesInten : 문자열	전일 대비 매출액 증감분을 출력
    - salesChange : 문자열	전일 대비 매출액 증감 비율을 출력
    - salesAcc : 문자열	누적매출액을 출력
    - audiCnt : 문자열	해당일의 관객수를 출력
    - audiInten : 문자열	전일 대비 관객수 증감분을 출력
    - audiChange : 문자열	전일 대비 관객수 증감 비율을 출력
    - audiAcc : 문자열	누적관객수를 출력합니다.
    - scrnCnt : 문자열	해당일자에 상영한 스크린수를 출력
    - showCnt : 문자열	해당일자에 상영된 횟수를 출력

In [84]:
import pandas as pd
import numpy as np
import time

df = pd.read_csv('../data/boxoffice_20_23.csv')
data = pd.read_csv('../data/boxoffice_15_19.csv')

In [85]:
concat_df = pd.concat([data,df], axis = 0)

In [86]:
df = concat_df.copy()

In [87]:
df['current_date']

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
14605    2023-12-31
14606    2023-12-31
14607    2023-12-31
14608    2023-12-31
14609    2023-12-31
Name: current_date, Length: 32870, dtype: object

In [88]:
movie_codes = df['movieCd'].drop_duplicates().tolist()

### 영화 상세정보 크롤링

In [9]:
# import requests
# import pandas as pd

# key = 'b713496d3bb846528153ba2985029132'
# movieCd = 20182530
# url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
# response = requests.get(url)
# r_data = response.json()

In [10]:
import requests
import pandas as pd

key = 'b713496d3bb846528153ba2985029132'
movie_codes = df['movieCd'].drop_duplicates().tolist()[0:2]

info_dfs = []

for movieCd in movie_codes:
    url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
    response = requests.get(url)
    r_data = response.json()

    # 응답 선택, 값이 없으면 none
    selected_data = {
        'movieCd': r_data['movieInfoResult']['movieInfo'].get('movieCd', None),
        'movieNm': r_data['movieInfoResult']['movieInfo'].get('movieNm', None),
        'showTm': r_data['movieInfoResult']['movieInfo'].get('showTm', None),
        'prdtYear': r_data['movieInfoResult']['movieInfo'].get('prdtYear', None),
        'openDt': r_data['movieInfoResult']['movieInfo'].get('openDt', None),
        'prdtStatNm': r_data['movieInfoResult']['movieInfo'].get('prdtStatNm', None),
        'typeNm': r_data['movieInfoResult']['movieInfo'].get('typeNm', None),
        'nationNm': r_data['movieInfoResult']['movieInfo']['nations'][0].get('nationNm', None),
        'genreNm': r_data['movieInfoResult']['movieInfo']['genres'][0].get('genreNm', None) if r_data['movieInfoResult']['movieInfo']['genres'] else None,
        'director': r_data['movieInfoResult']['movieInfo']['directors'][0].get('peopleNm', None) if r_data['movieInfoResult']['movieInfo']['directors'] else None,
        'actor': ', '.join(actor['peopleNm'] for actor in r_data['movieInfoResult']['movieInfo']['actors']) if r_data['movieInfoResult']['movieInfo']['actors'] else None,
        'showTypeGroupNm': ', '.join(stype['showTypeGroupNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
        'showTypeNm': ', '.join(stype['showTypeNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
        'audits': ', '.join(audit['watchGradeNm'] for audit in r_data['movieInfoResult']['movieInfo'].get('audits', [])) if r_data['movieInfoResult']['movieInfo'].get('audits') else None,
        
    }

    info_df = pd.DataFrame([selected_data])
    info_dfs.append(info_df)


total_info_df = pd.concat(info_dfs, ignore_index=True)
total_info_df.head()


,movieCd,movieNm,showTm,prdtYear,openDt,prdtStatNm,typeNm,nationNm,genreNm,director,actor,showTypeGroupNm,showTypeNm,audits
0,20137048,국제시장,126,2014,20141217,개봉,장편,한국,드라마,윤제균,"황정민, 김윤진, 오달수, 정진영, 장영남, 라미란, 김슬기, 정영기, 맹세창, 박...","필름, 2D, 2D, 2D","필름, 디지털, 디지털 영문자막, 디지털 가치봄",12세이상관람가
1,20143642,테이큰 3,111,2014,20150101,개봉,장편,미국,액션,올리비에 메가턴,"리암 니슨, 팜케 얀센, 매기 그레이스, 존 그리스, 포레스트 휘테커","필름, 2D, 4D","필름, 디지털, 4D",15세이상관람가


In [11]:
%%time
import requests

key = '75b69dc273dc0e8cef4e9cfb4736799f'
movie_codes = df['movieCd'].drop_duplicates().tolist()

info_dfs = []

for movieCd in movie_codes:
    try:
        url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movieCd}'
        response = requests.get(url)
        r_data = response.json()

        # 필요한 필드 선택
        selected_data = {
            'movieCd': r_data['movieInfoResult']['movieInfo'].get('movieCd', None),
            'movieNm': r_data['movieInfoResult']['movieInfo'].get('movieNm', None),
            'showTm': r_data['movieInfoResult']['movieInfo'].get('showTm', None),
            'prdtYear': r_data['movieInfoResult']['movieInfo'].get('prdtYear', None),
            'openDt': r_data['movieInfoResult']['movieInfo'].get('openDt', None),
            'prdtStatNm': r_data['movieInfoResult']['movieInfo'].get('prdtStatNm', None),
            'typeNm': r_data['movieInfoResult']['movieInfo'].get('typeNm', None),
            'nationNm': r_data['movieInfoResult']['movieInfo']['nations'][0]['nationNm'] if r_data['movieInfoResult']['movieInfo'].get('nations') else None,
            'genreNm': r_data['movieInfoResult']['movieInfo']['genres'][0]['genreNm'] if r_data['movieInfoResult']['movieInfo'].get('genres') else None,
            'director': r_data['movieInfoResult']['movieInfo']['directors'][0]['peopleNm'] if r_data['movieInfoResult']['movieInfo'].get('directors') else None,
            'actor': ', '.join(actor['peopleNm'] for actor in r_data['movieInfoResult']['movieInfo'].get('actors', [])),
            'showTypeGroupNm': ', '.join(stype['showTypeGroupNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'showTypeNm': ', '.join(stype['showTypeNm'] for stype in r_data['movieInfoResult']['movieInfo'].get('showTypes', [])) if r_data['movieInfoResult']['movieInfo'].get('showTypes') else None,
            'audits': ', '.join(audit['watchGradeNm'] for audit in r_data['movieInfoResult']['movieInfo'].get('audits', [])) if r_data['movieInfoResult']['movieInfo'].get('audits') else None
        }

        info_df = pd.DataFrame([selected_data])
        info_dfs.append(info_df)
    except Exception as e:
        print(f"Error occurred while processing movie code {movieCd}: {e}")

total_info_df = pd.concat(info_dfs, ignore_index=True)
total_info_df.head()


KeyboardInterrupt: 

In [13]:
#total_info_df.to_csv('../data/boxoffice_info_15_23.csv',index = False, encoding = 'utf-8-sig')

In [108]:
total_info_df = pd.read_csv('../data/boxoffice_info_15_23.csv')

In [109]:
total = pd.merge(df,total_info_df, on= ['movieCd','movieNm'], how = 'left')

In [110]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33523 entries, 0 to 33522
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rnum             33523 non-null  int64  
 1   rank             33523 non-null  int64  
 2   rankInten        33523 non-null  int64  
 3   rankOldAndNew    33523 non-null  object 
 4   movieCd          33523 non-null  object 
 5   movieNm          33523 non-null  object 
 6   openDt_x         33523 non-null  object 
 7   salesAmt         33523 non-null  int64  
 8   salesShare       33523 non-null  float64
 9   salesInten       33523 non-null  int64  
 10  salesChange      33523 non-null  float64
 11  salesAcc         33523 non-null  int64  
 12  audiCnt          33523 non-null  int64  
 13  audiInten        33523 non-null  int64  
 14  audiChange       33523 non-null  float64
 15  audiAcc          33523 non-null  int64  
 16  scrnCnt          33523 non-null  int64  
 17  showCnt     

In [111]:
total[['openDt_x','openDt_y']] # 년도별 형식이 다름

,openDt_x,openDt_y
0,2014-12-17,NaN
1,2015-01-01,NaN
2,2014-12-31,NaN
3,2014-11-27,NaN
4,2014-12-24,NaN
...,...,...
33518,2023-12-27,20231227.0
33519,2023-12-13,20231213.0
33520,2023-11-29,20231129.0
33521,2023-12-27,20231227.0


In [114]:
# 날짜 형식으로 변환
total['openDt_y'] = pd.to_datetime(total['openDt_y'], format='%Y%m%d')

In [115]:
total['openDt_x'].fillna(total['openDt_y'], inplace=True)

In [116]:
total.drop(columns = 'openDt_y', inplace = True)
total.rename(columns = {'openDt_x' : 'openDt'}, inplace = True)

In [117]:
total.openDt

0        2014-12-17
1        2015-01-01
2        2014-12-31
3        2014-11-27
4        2014-12-24
            ...    
33518    2023-12-27
33519    2023-12-13
33520    2023-11-29
33521    2023-12-27
33522    2023-12-27
Name: openDt, Length: 33523, dtype: object

In [118]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33523 entries, 0 to 33522
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rnum             33523 non-null  int64  
 1   rank             33523 non-null  int64  
 2   rankInten        33523 non-null  int64  
 3   rankOldAndNew    33523 non-null  object 
 4   movieCd          33523 non-null  object 
 5   movieNm          33523 non-null  object 
 6   openDt           33523 non-null  object 
 7   salesAmt         33523 non-null  int64  
 8   salesShare       33523 non-null  float64
 9   salesInten       33523 non-null  int64  
 10  salesChange      33523 non-null  float64
 11  salesAcc         33523 non-null  int64  
 12  audiCnt          33523 non-null  int64  
 13  audiInten        33523 non-null  int64  
 14  audiChange       33523 non-null  float64
 15  audiAcc          33523 non-null  int64  
 16  scrnCnt          33523 non-null  int64  
 17  showCnt     

In [119]:
total.current_date

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
33518    2023-12-31
33519    2023-12-31
33520    2023-12-31
33521    2023-12-31
33522    2023-12-31
Name: current_date, Length: 33523, dtype: object

In [120]:
total.to_csv('../data/boxoffice_15_23_total.csv',index = False, encoding = 'utf-8-sig')